In [3]:
import pandas as pd
import numpy as np

task = 'Fodors-Zagat'
df = pd.read_csv('data/'+task +'/train.csv')


cols = []
for col in df.columns:
    col = col.replace('left_','').replace('right_','')
    if col not in cols:
        if col not in ['id','label']:
            cols.append(col)

T = len(cols)*len(df)

frac = 0.9



In [108]:
import random
import pandas as pd
import numpy as np


def clac_p_berno(val):
    MISS_COND = {1 : 0.8, 0: 0.2}
    x = MISS_COND[val['label']]
    return (np.arctan(x)/np.arctan(len(MISS_COND)))+ random.uniform(-0.05, 0.05)



def inject_MCAR(df_in,frac,cols):
    df = df_in.copy()

    n_missing = int(frac * len(df) * len(cols))
    missing_indices = []
    while len(missing_indices) < n_missing:
        row = np.random.randint(0, df.shape[0])
        col = np.random.randint(0, len(cols))
        missing_indices.append(np.array([row, col]))
        missing_indices  = list(np.unique(missing_indices,axis = 0))
    for row, col in missing_indices:
        df.loc[row,'left_'+cols[col]] = np.nan

    n_missing = int(frac * len(df) * len(cols))
    missing_indices = []
    while len(missing_indices) < n_missing:
        row = np.random.randint(0, df.shape[0])
        col = np.random.randint(0, len(cols))
        missing_indices.append(np.array([row, col]))
        missing_indices  = list(np.unique(missing_indices,axis = 0))
    for row, col in missing_indices:
        df.loc[row,'right_'+cols[col]] = np.nan
    return df



def inject_MAR(df_in,frac,cols):
    df = df_in.copy()
    left_cols = ['left_'+col for col in cols]
    right_cols = ['right_'+col for col in cols]

    n_missing = int(frac * len(df) * len(cols))
    number_nans = df.isna()[left_cols].sum().sum()
    while number_nans < n_missing:
        col = np.random.randint(0, len(cols))
        row = np.random.randint(0, df.shape[0])
        p_berno  = clac_p_berno(df.loc[row])
        action = [np.random.rand() < p_berno for _ in range(1)][0]
        if action : 
            df.loc[row,'left_'+cols[col]] = np.nan
        number_nans = df.isna()[left_cols].sum().sum()

    n_missing = int(frac * len(df) * len(cols))
    number_nans = df.isna()[right_cols].sum().sum()
    while number_nans < n_missing:
        col = np.random.randint(0, len(cols))
        row = np.random.randint(0, df.shape[0])
        p_berno  = clac_p_berno(df.loc[row])
        action = [np.random.rand() < p_berno for _ in range(1)][0]
        if action : 
            df.loc[row,'right_'+cols[col]] = np.nan
        number_nans = df.isna()[right_cols].sum().sum()

    return df








def clac_p_berno_MNAR(val,COL):
    MISS_COND = {1 : 0.8, 0: 0.2}
    x = MISS_COND[val['label']] 
    x+= (hash(val[COL])% 100)/100
    return (np.arctan(x)/np.arctan(len(MISS_COND)+100)) + random.uniform(-0.05, 0.05)


def inject_MNAR(df_in,frac,cols):
    df = df_in.copy()
    left_cols = ['left_'+col for col in cols]
    right_cols = ['right_'+col for col in cols]

    n_missing = int(frac * len(df) * len(cols))
    number_nans = df.isna()[left_cols].sum().sum()
    while number_nans < n_missing:
        col = np.random.randint(0, len(cols))
        row = np.random.randint(0, df.shape[0])
        p_berno  = clac_p_berno_MNAR(df.loc[row],'left_'+cols[col])
        action = [np.random.rand() < p_berno for _ in range(1)][0]
        if action : 
            df.loc[row,'left_'+cols[col]] = np.nan
        number_nans = df.isna()[left_cols].sum().sum()
    
    n_missing = int(frac * len(df) * len(cols))
    number_nans = df.isna()[right_cols].sum().sum()
    while number_nans < n_missing:
        col = np.random.randint(0, len(cols))
        row = np.random.randint(0, df.shape[0])
        p_berno  = clac_p_berno_MNAR(df.loc[row],'right_'+cols[col])
        action = [np.random.rand() < p_berno for _ in range(1)][0]
        if action : 
            df.loc[row,'right_'+cols[col]] = np.nan
        number_nans = df.isna()[right_cols].sum().sum()
    
    return df





# import pandas as pd
# df = pd.read_csv('data/'+task +'/train.csv')
# frac = 0.1
